In [1]:
%%capture
%pip install pulp

In [2]:
from pulp import *

| Modalidade | Propostas Recebidas | Capital Necessário (Média por cliente) | Taxa de Juros (%) |
|------------|---------------------|----------------------------------------|-------------------|
| Capital de Giro | 1400 | 18000 | 2.13 |
| Cheque Especial | 600 | 10000 | 8.16 |
| Crédito Pessoal | 1600 | 15000 | 6.43 |
| Crédito Pessoal Consignado | 6000 | 12500 | 2.36 |
| Financiamento Imobiliário | 3400 | 13425 | 0.81 |
| Aquisição de Veículos | 3400 | 8500 | 1.86 |

In [3]:
modelo = LpProblem('Maximizar_Lucro_Modalidades_Credito', LpMaximize)

x1 = LpVariable('Qnt_Clientes_Capital_de_Giro', lowBound=0)
x2 = LpVariable('Qnt_Clientes_Cheque_Especial', lowBound=0)
x3 = LpVariable('Qnt_Clientes_Crédito_Pessoal', lowBound=0)
x4 = LpVariable('Qnt_Clientes_Crédito_Pessoal_Consignado', lowBound=0)
x5 = LpVariable('Qnt_Clientes_Financiamento_Imobiliário', lowBound=0)
x6 = LpVariable('Qnt_Clientes_Aquisição_de_Veículos', lowBound=0)

### Objetivo (Maximização do Lucro gerado por cada Modalidade)

Lucro do Cliente Individual = Taxas de Juros (%) * Capital Necessário Médio * Quantidade de Clientes

In [4]:
modelo +=\
	(2.13 / 100 * 18_000 * x1) +\
	(8.16 / 100 * 10_000 * x2) +\
	(6.43 / 100 * 15_000 * x3) +\
	(2.36 / 100 * 12_500 * x4) +\
	(0.81 / 100 * 13_425 * x5) +\
	(1.86 / 100 * 8_500 * x6), 'Lucro_Total'

### Restrições do Modelo

#### Restringindo por quantidade máxima de clientes

In [5]:
MAXIMO_CLIENTES = 6000

In [6]:
modelo += x1 + x2 + x3 + x4 + x5 + x6 <= MAXIMO_CLIENTES, 'Total_de_Clientes_Processados'

#### Restringindo por total de número de clientes interessados

In [7]:
for var in [
	(x1 <= 1400, 'Máximo de clientes para Capital de Giro'),
	(x2 <= 600, 'Máximo de clientes para Cheque Especial'),
	(x3 <= 1600, 'Máximo de clientes para Crédito Pessoal'),
	(x4 <= 6000, 'Máximo de clientes para Crédito Pessoal Consignado'),
	(x5 <= 3400, 'Máximo de clientes para Financiamento Imobiliário'),
	(x6 <= 3400, 'Máximo de clientes para Aquisição de Veículos'),
]:
	modelo += var

#### Restringindo por total de capital disponível

In [8]:
CAPITAL_MAXIMO = 125_000_000

In [9]:
modelo +=\
	18_000 * x1 +\
	10_000 * x2 +\
	15_000 * x3 +\
	12_500 * x4 +\
	13_425 * x5 +\
	8_500 * x6 <= CAPITAL_MAXIMO, 'Capital_Total_Disponível'

#### Restringindo por número mínimo de clientes que precisamos pegar de cada modalidade

In [10]:
AMOSTRA_MINIMA_POSSIVEL = 400

In [11]:
for var in [
	(x1 >= AMOSTRA_MINIMA_POSSIVEL, 'Mínimo de clientes de Capital de Giro'),
	(x2 >= AMOSTRA_MINIMA_POSSIVEL, 'Mínimo de clientes de Cheque Especial'),
	(x3 >= AMOSTRA_MINIMA_POSSIVEL, 'Mínimo de clientes de Crédito Pessoal'),
	(x4 >= AMOSTRA_MINIMA_POSSIVEL, 'Mínimo de clientes de Crédito Pessoal Consignado'),
	(x5 >= AMOSTRA_MINIMA_POSSIVEL, 'Mínimo de clientes de Financiamento Imobiliário'),
	(x6 >= AMOSTRA_MINIMA_POSSIVEL, 'Mínimo de clientes de Aquisição de Veículos'),
]:
	modelo += var

### Visualização dos resultados

In [12]:
modelo.solve(PULP_CBC_CMD(msg=False))

1

In [13]:
print(f"Clientes de Capital de Giro (x1): {x1.varValue}")
print(f"Clientes de Cheque Especial (x2): {x2.varValue}")
print(f"Clientes de Crédito Pessoal (x3): {x3.varValue}")
print(f"Clientes de Crédito Pessoal Consignado (x4): {x4.varValue}")
print(f"Clientes de Financiamento Imobiliário (x5): {x5.varValue}")
print(f"Clientes de Aquisição de Veículos (x6): {x6.varValue}")

Clientes de Capital de Giro (x1): 1400.0
Clientes de Cheque Especial (x2): 600.0
Clientes de Crédito Pessoal (x3): 1600.0
Clientes de Crédito Pessoal Consignado (x4): 1600.0
Clientes de Financiamento Imobiliário (x5): 400.0
Clientes de Aquisição de Veículos (x6): 400.0


In [14]:
f"Lucro Total: {modelo.objective.value()}"

'Lucro Total: 3148297.0'